In [ ]:
import numpy as np
from scipy.stats import norm, ncx2
from scipy.optimize import root, minimize

### 11.2.4 Quadrature Methods in Practice: Digital Options Prices in Black-Scholes vs. Bachelier Model

In this coding example we show how we could use quadrature to price a digital option with a specified risk-neutral density:

In [ ]:
def price_digital_call_quad(S_0, K, r, T, density_func, b, N):
    """Using left riemann sum"""
    width = (b - K) / N
    nodes = np.arange(K, b, width)
    areas = [width * density_func(node) for node in nodes]
    px = np.exp(-r * T) * sum(areas)
    return px


# example usage
S_0 = 100    # initial stock price
K = 105      # strike price
r = 0.05     # risk-free interest rate
T = 1        # time to maturity in years
b = 150      # upper bound for integration
N = 1000     # number of intervals

# standard normal density function
density_func = norm(loc=S_0, scale=20).pdf

# calculate the price of the digital call option
price = price_digital_call_quad(S_0, K, r, T, density_func, b, N)
print(f"Price of the digital call option: {price}")

Price of the digital call option: 0.37621058444634486


### 11.3.14 FFT Pricing in Practice: Sensitivity to Technique Parameters

In the following coding example we detail how the FFT techniques introduced in this chapter can be used to value European options using the Heston stochastic volatility:

In [ ]:
def heston_characteristic_eqn(u, sigma, k, p, s_0, r, t, theta, v_0):

    lambd = np.sqrt((sigma**2) * ((u**2) + 1j * u) + (k - 1j * p * sigma * u) ** 2)
    omega_numerator = np.exp(
        1j * u * np.log(s_0)
        + 1j * u * (r) * t
        + (1 / (sigma**2)) * k * theta * t * (k - 1j * p * sigma * u)
    )
    omega_denominator = (
        np.cosh(0.5 * lambd * t)
        + (1 / lambd) * (k - 1j * p * sigma * u) * np.sinh(0.5 * lambd * t)
    ) ** ((2 * k * theta) / (sigma**2))
    phi = (omega_numerator / omega_denominator) * np.exp(
        -((u**2 + 1j * u) * v_0)
        / (lambd * (1 / np.tanh(0.5 * lambd * t)) + (k - 1j * p * sigma * u))
    )
    return phi

def calc_fft_heston_call_prices(
    alpha, N, delta_v, sigma, k, p, s_0, r, t, theta, v_0, K=None
):
    # delta is the indicator function
    delta = np.zeros(N)
    delta[0] = 1
    delta_k = (2 * np.pi) / (N * delta_v)
    if K == None:
        # middle strike is at the money
        beta = np.log(s_0) - delta_k * N * 0.5
    else:
        # middle strike is K
        beta = np.log(K) - delta_k * N * 0.5
    k_list = np.array([(beta + (i - 1) * delta_k) for i in range(1, N + 1)])
    v_list = np.arange(N) * delta_v
    # building fft input vector
    x_numerator = np.array(
        [((2 - delta[i]) * delta_v) * np.exp(-r * t) for i in range(N)]
    )
    x_denominator = np.array(
        [2 * (alpha + 1j * i) * (alpha + 1j * i + 1) for i in v_list]
    )
    x_exp = np.array([np.exp(-1j * (beta) * i) for i in v_list])
    x_list = (
        (x_numerator / x_denominator)
        * x_exp
        * np.array(
            [
                heston_characteristic_eqn(
                    i - 1j * (alpha + 1), sigma, k, p, s_0, r, t, theta, v_0
                )
                for i in v_list
            ]
        )
    )
    # fft output
    y_list = np.fft.fft(x_list)
    # recovering prices
    prices = np.array(
        [
            (1 / np.pi)
            * np.exp(-alpha * (beta + (i - 1) * delta_k))
            * np.real(y_list[i - 1])
            for i in range(1, N + 1)
        ]
    )
    return prices, np.exp(k_list)


# example usage
alpha = 1.5
N = 1024
delta_v = 0.25
sigma = 0.2
k = 2.0
p = -0.7
s_0 = 100
r = 0.05
t = 1
theta = 0.1
v_0 = 0.1
K = 100

prices, strikes = calc_fft_heston_call_prices(alpha, N, delta_v, sigma, k, p, s_0, r, t, theta, v_0, K)

print("Strikes:")
print(strikes)
print("\nCall Prices:")
print(prices)

Strikes:
[3.48734236e-04 3.57399364e-04 3.66279797e-04 ... 2.66396089e+07
 2.73015331e+07 2.79799043e+07]

Call Prices:
[9.99996682e+01 9.99996598e+01 9.99996515e+01 ... 4.24113809e-15
 4.24113777e-15 4.24113744e-15]


### 11.4.4 Implied Volatility in Practice: Volatility Skew for VIX Options

In the following coding example, we show to calculate an implied volatility for a given option in the Black-Scholes model using a one-dimensional root finding algorithm:

In [ ]:
def bs_call(S_0, K, T, sigma, r):
    d_1 = (np.log(S_0 / K) + T * (r + (sigma**2) / 2)) / (sigma * np.sqrt(T))
    d_2 = d_1 - sigma * np.sqrt(T)
    return S_0 * norm.cdf(d_1) - K * np.exp(-r * T) * norm.cdf(d_2)

def get_implied_vol_bs(S_0, K, T, r, observed_px, initial_guess):
    root_fn = lambda x: bs_call(S_0, K, T, x, r) - observed_px
    return root(root_fn, initial_guess)["x"][0]


# example usage
S_0 = 100              # initial stock price
K = 105                # strike price
T = 1                  # time to maturity in years
r = 0.05               # risk-free interest rate
observed_px = 10       # observed option price
initial_guess = 0.2    # initial guess for implied volatility

# calculate the Black-Scholes call price
sigma = 0.25           # volatility
call_price = bs_call(S_0, K, T, sigma, r)
print(f"Black-Scholes Call Price: {call_price}")

# calculate the implied volatility
implied_vol = get_implied_vol_bs(S_0, K, T, r, observed_px, initial_guess)
print(f"Implied Volatility: {implied_vol}")


Black-Scholes Call Price: 10.00220211715488
Implied Volatility: 0.24994433396047658


### 11.5.7 Minimum Variance Portfolios in Practice: Stock & Bond Minimum Variance Portfolio Weights

In this coding example, we show how to find a minumum variance portfolio for two assets using equations (11.97) and (11.98):

In [ ]:
def min_var_portfolio(sigma_1, sigma_2, rho):
    numerator = sigma_2**2 - sigma_1 * sigma_2 * rho
    denominator = sigma_1**2 + sigma_2**2 - 2 * sigma_1 * sigma_2 * rho
    w_1 = numerator / denominator
    return (w_1, 1 - w_1)

print(min_var_portfolio(0.25, 0.3, -0.4))

(0.5647058823529412, 0.43529411764705883)


To verify our result, we also show how the same problem can be computed using Python's built-in optimization libraries:

In [ ]:
# function to minimize
def portfolio_variance(weights, cov_mat):
    return weights.T @ cov_mat @ weights

def min_var_portfolio_opt(cov_mat):
    guess = np.array([1 / len(cov_mat)] * len(cov_mat))      # equal weights guess
    cons = {"type": "eq", "fun": lambda x: np.sum(x) - 1}    # sum of weights equal to 1
    return minimize(
        portfolio_variance,
        x0=guess,
        args=(cov_mat),
        constraints=cons,
        tol=1e-8,
        method="SLSQP",
    )["x"]

correlation = -0.4
sigma_1 = 0.25
sigma_2 = 0.3
cov_mat = np.array(
    [
        [sigma_1**2, correlation * sigma_1 * sigma_2],
        [correlation * sigma_1 * sigma_2, sigma_2**2],
    ]
)

print(min_var_portfolio_opt(cov_mat))

[0.56470589 0.43529411]


### 11.6.8 Calibration in Practice: BRLJPY Currency Options

In this coding example we detail how a volatility surface can be calibrated to the CEV model:


In [ ]:
# reuse cev_call() from 10.1.5
def cev_call(S_0, K, T, sigma, beta, r):
    v = 1 / (2 * (1 - beta))
    x_1 = 4 * (v**2) * (K ** (1 / v)) / ((sigma**2) * T)
    x_2 = 4 * (v**2) * ((S_0 * np.exp(r * T)) ** (1 / v)) / ((sigma**2) * T)
    kappa_1 = 2 * v + 2
    kappa_2 = 2 * v
    lambda_1 = x_2
    lambda_2 = x_1
    return np.exp(-r * T) * (
        (S_0 * np.exp(r * T) * (1 - ncx2.cdf(x_1, kappa_1, lambda_1)))
        - K * ncx2.cdf(x_2, kappa_2, lambda_2)
    )

In [ ]:
def sum_squares_cev(beta, S_0, sigma, r, call_prices, strikes, expiries):
    sum = 0
    for price, strike, expiry in zip(call_prices, strikes, expiries):
        sum += (
            price - cev_call(S_0, strike, expiry, sigma, r, beta)
        ) ** 2
    return sum


def find_beta_sigma_cev(
    S_0,
    r,
    call_prices,
    strikes,
    expiries,
    guess=[0.9, 0.4],
    bounds=((0.001, None), (0.001, None)),
    tol=1e-10,
):
    """
    call_prices, strikes, and expiries are arrays of equal length with each index corresponding to one option
    For example, if the first elements of each array are 10, 100, and 1 respectively, this corresponds to an option
    with price 10, strike 100, and 1 year to expiry
    """
    # first element is beta, second element is sigma
    unique_expiries = np.unique(expiries)
    calibrated_beta_sigma = {}  # this is where the results will go for each expiry
    for expiry in unique_expiries:
        # how many times this expiry appears
        expiries_list = expiry * np.ones(np.sum(expiries == expiry))
        # where it appears
        indices = np.where(expiries == expiry)[0]
        # what are the strikes for this expiry
        strikes_for_expiry = np.array(strikes)[indices]
        # what are the market prices for this expiry
        prices_for_expiry = np.array(call_prices)[indices]
        # the function to be minimized
        opt_func = lambda x: sum_squares_cev(
            x[0], S_0, x[1], r, prices_for_expiry, strikes_for_expiry, expiries_list
        )
        # calibrated values
        beta, sigma = minimize(opt_func, guess, bounds=bounds, method="SLSQP", tol=tol)[
            "x"
        ]
        # saving the results
        calibrated_beta_sigma[expiry] = [beta, sigma]
    return calibrated_beta_sigma


# example usage
S_0 = 100                           # initial stock price
r = 0.05                            # risk-free interest rate
call_prices = [10, 12, 14, 16, 18]  # example call prices
strikes = [90, 95, 100, 105, 110]   # example strikes
expiries = [0.5, 0.5, 1, 1, 1.5]    # example expiries

# find beta and sigma for the CEV model
calibrated_params = find_beta_sigma_cev(S_0, r, call_prices, strikes, expiries)
print("Calibrated beta and sigma for each expiry:")
print(calibrated_params)

Calibrated beta and sigma for each expiry:
{np.float64(0.5): [np.float64(0.07921237046687302), np.float64(0.4)], np.float64(1.0): [np.float64(0.0010000000000000067), np.float64(32.156173503202126)], np.float64(1.5): [np.float64(0.0010075141948618915), np.float64(0.4)]}
